In [0]:
cd '/content/drive/My Drive/PD/cis-pd'

/content/drive/My Drive/PD/cis-pd


In [0]:
!pip install hmmlearn
from hmmlearn import hmm
import numpy as np
import pickle
import os

     |████████████████████████████████| 368kB 3.2MB/s 


In [0]:
'''
Read HMM model for transforming data sequence into state sequence.
Read memo for splitting data into the original length.
'''

with open("model_Total_50.pkl", "rb") as file: 
  m = pickle.load(file)
with open("cis-pd-memo.pkl", "rb") as file: 
  memo = pickle.load(file)

In [0]:
m

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, init_params='', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=50, n_iter=1, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=True)

In [0]:
'''
Read the name of file in the specified directory.
'''

directory = 'outputA_Total/'
l = []
name = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      name.append(filename[:-4])

In [0]:
'''
Transorm data sequence to state sequence.
Then generate the transition matrix for each state sequence, and store it in all_tran.
'''

all_tran = {}
counter = 0

c_state = m.n_components
for n in name:
  cur = 0
  k = np.loadtxt(directory+n+'.csv')
  for t in memo[int(n)]['train']+memo[int(n)]['test']:
    seq = m.predict(k[cur:cur+t])
    cur += (t+1)
    tran = np.zeros((c_state,c_state))
    for ts in range(t-1):
      tran[seq[ts]-1][seq[ts-1]-1] += 1
    tran = np.where(tran == 0,0.1,tran)

    tran = tran/tran.sum(axis=1).reshape(-1,1)
    all_tran[counter] = tran
    counter+=1

'''
Find the stationary distribution of each transition matrix, and store it in all_HMM.
'''

all_HMM = np.zeros((len(all_tran),c_state*2))

for i in range(len(all_tran)):
  tran = all_tran[i]
  d,v = np.linalg.eig(tran.T)
  pos_zero = np.argmin(abs(d-1))
  if np.round(d[pos_zero],6) != 1:
    print('{} has no eigenvalue equal to one'.format(i))
  
  p = v[pos_zero]
  p /= sum(p)

  all_HMM[i,:c_state] = p

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: ComplexWarning: Casting complex values to real discards the imaginary part


In [0]:
'''
Split all_HMM into train_HMM and test_HMM. Note that, both of them are used when building HMM, so the data is needed to split for latter training.
Also, I concatenate the mean value of each patient at this stage.
'''

cur = 0
test_HMM = np.zeros((1,2*c_state))
train_HMM = np.zeros((1,2*c_state))
train_train_HMM = np.zeros((1,2*c_state))
train_test_HMM = np.zeros((1,2*c_state))
for n in name:
  l_train = len(memo[int(n)]['train'])
  l_test = len(memo[int(n)]['test'])

  all_HMM[cur:l_train+l_test+cur,c_state:] = np.tile(all_HMM[cur:l_train+l_test+cur,:c_state].mean(axis = 0),(l_train+l_test,1))
  train_HMM = np.concatenate((train_HMM,all_HMM[cur:l_train+cur]))
  # s = np.arange(l_train)
  # np.random.shuffle(s)
  # m = l_train//4
  # train_train_HMM = np.concatenate((train_train_HMM,all_HMM[cur:l_train+cur][m:]))
  # train_test_HMM = np.concatenate((train_test_HMM,all_HMM[cur:l_train+cur][:m]))
  
  cur += l_train
  test_HMM = np.concatenate((test_HMM,all_HMM[cur:cur+l_test]))
  cur += l_test

In [0]:
train_HMM.shape

(1859, 100)

In [0]:
test_HMM.shape

(619, 100)

In [0]:
'''
Save the train_HMM and test_HMM for next step.
'''

with open("train_Total_HMM_1.pkl", "wb") as file: 
  pickle.dump(train_HMM[1:],file)

with open("test_Total_HMM_1.pkl", "wb") as file: 
  pickle.dump(test_HMM[1:],file)